In [5]:
import nltk 

with open('script.txt') as script:
    lines = script.readlines()

sentences = []

for line in lines:
    modified_line = ''
    for ch in line:
        ch = ch.lower()
        modified_line += ch
    # Remove unnecessary characters and whitespace
    modified_line = modified_line.replace('...', '').replace('\n', '').replace('!', '').replace('.', '').replace('\ufeff', '')
    if modified_line.strip():  # Check if the line is not empty after stripping whitespace
        sentences.append(modified_line)



from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser(url="http://localhost:9000")


CorenlpGrammarTree = next(parser.raw_parse(line))

import nltk

def tree_to_cfg(tree):
    cfg = {}

    def traverse(node):
        if isinstance(node, str):  # Terminal node
            return [node]
        else:
            children = [traverse(child) for child in node]
            non_terminal = node.label()
            if non_terminal not in cfg:
                cfg[non_terminal] = []
            cfg[non_terminal].extend(children)
            return [non_terminal]

    traverse(tree)
    return cfg

# Parse tree in the string format
parse_tree_str = str(CorenlpGrammarTree)

# Convert string to nltk.Tree object
parse_tree = nltk.Tree.fromstring(parse_tree_str)

# Convert the parse tree to CFG
cfg = tree_to_cfg(parse_tree)

# Print the CFG rules
for non_terminal, expansions in cfg.items():
    print(non_terminal + " -> " + " | ".join([" ".join(expansion) for expansion in expansions]))

NNP -> Meow
. -> .
NP -> NNP | .
ROOT -> NP


In [6]:
import nltk
from nltk.parse.corenlp import CoreNLPParser

import nltk
from nltk.tokenize import sent_tokenize
import contractions
import re

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')

# Function to preprocess a sentence
def preprocess_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Expand contractions
    sentence = contractions.fix(sentence)
    # Remove non-alphanumeric characters and extra whitespace
    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Remove punctuation marks
    sentence = re.sub(r'[.,?!]', '', sentence)
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

# Open the file and preprocess the corpus
with open('script.txt') as script:
    text = script.read()

# Remove extra newline characters
text = text.replace('\n', ' ')

# Tokenize into sentences
sentences = sent_tokenize(text)

# Preprocess each sentence and store in a list
preprocessed_sentences = [preprocess_sentence(sentence) for sentence in sentences]



# Initialize CoreNLPParser
parser = CoreNLPParser(url="http://localhost:9000")

# Initialize an empty CFG
combined_cfg = {}

# Process each sentence and merge parse trees into a single CFG
for lines in preprocessed_sentences:
    CorenlpGrammarTree = next(parser.raw_parse(lines))

    def tree_to_cfg(tree):
        cfg = {}

        def traverse(node):
            if isinstance(node, str):  # Terminal node
                return [node]
            else:
                children = [traverse(child) for child in node]
                non_terminal = node.label()
                if non_terminal not in cfg:
                    cfg[non_terminal] = []
                cfg[non_terminal].extend(children)
                return [non_terminal]

        traverse(tree)
        return cfg

    # Convert the parse tree to CFG
    cfg = tree_to_cfg(CorenlpGrammarTree)

    # Merge the CFG of the current sentence into the combined CFG
    for non_terminal, expansions in cfg.items():
        if non_terminal not in combined_cfg:
            combined_cfg[non_terminal] = []
        combined_cfg[non_terminal].extend(expansions)

# Print the combined CFG rules
for non_terminal, expansions in combined_cfg.items():
    unique_expansions = set(tuple(expansion) for expansion in expansions)
    print(non_terminal + " -> " + " | ".join([" ".join(expansion) for expansion in unique_expansions]))

[nltk_data] Downloading package punkt to /home/rosa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


IN -> at | into | till | than | that | with | under | since | before | from | if | behind | until | for | though | by | without | because | through | once | save | in | beyond | on | after | like | along | over | between | about | as | out | to | during | of
DT -> those | all | an | every | a | that | this | some | each | the | both | no | any | these
NNS -> lollipops | everythings | ways | huevos | chupacabra | backpacks | brothers | curtains | months | goners | scars | chances | gentlemen | soldiers | days | cheers | familys | muffins | feelings | dukes | frijoles | teenagers | boots | nobodys | brains | sizes | legends | friends | comandantes | seconds | times | games | thanks | ladies | onions | moves | names | clouds | lives | twotimes | cahoots | eyes | dogs | clocks | softpaws | thieves | flaws | socks | women | hands | candlesticks | children | hamhocks | kids | goggles | people | piggies | mamas | adventures | feet | outlaws | orphans | paws | ambushes | churches | giants | eg

In [15]:
import random

# Functie om een willekeurige zin te genereren met behulp van CFG
def genereer_zin(cfg, symbool):
    if symbool not in cfg:
        return symbool  # Terminaal symbool, retourneer het symbool zelf
    uitbreidingen = cfg[symbool]
    gekozen_uitbreiding = random.choice(uitbreidingen)
    return ' '.join(genereer_zin(cfg, s) for s in gekozen_uitbreiding)

# Genereer 12 willekeurige zinnen met behulp van de gecombineerde CFG
random.seed(42)  # Zet een willekeurige zaad voor reproduceerbaarheid
for _ in range(12):
    zin = genereer_zin(combined_cfg, 'ROOT')  # Begin met het start symbool 'S'
    print(zin)

blue
hello
are
let
please
rosa
me
me
puss
beans
you
the


In [20]:
import nltk
from nltk.parse.corenlp import CoreNLPParser
from nltk.tokenize import sent_tokenize
import contractions
import re

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')

# Function to preprocess a sentence
def preprocess_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Expand contractions
    sentence = contractions.fix(sentence)
    # Remove non-alphanumeric characters and extra whitespace
    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Remove punctuation marks
    sentence = re.sub(r'[.,?!]', '', sentence)
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence.strip()

# Open the file and preprocess the corpus
with open('script.txt') as script:
    text = script.read()

# Remove extra newline characters
text = text.replace('\n', ' ')

# Tokenize into sentences
sentences = sent_tokenize(text)

# Preprocess each sentence and store in a list
preprocessed_sentences = [preprocess_sentence(sentence) for sentence in sentences]

# Initialize CoreNLPParser
parser = CoreNLPParser(url="http://localhost:9000")

# Function to convert tree to productions
def tree_to_productions(tree):
    productions = []

    def traverse(node):
        if isinstance(node, str):  # Terminal node
            return [node]
        else:
            children = [traverse(child) for child in node]
            non_terminal = node.label()
            production = (non_terminal, tuple(child[0] for child in children if child))
            productions.append(production)
            return [non_terminal]

    traverse(tree)
    return productions

# Initialize a set to store productions
productions_set = set()

# Process each sentence and collect productions
for line in preprocessed_sentences:
    CorenlpGrammarTree = next(parser.raw_parse(line))

    # Convert the parse tree to productions
    productions = tree_to_productions(CorenlpGrammarTree)

    # Add productions to the set
    productions_set.update(productions)

# Print the productions
for non_terminal, expansions in productions_set:
    print(f"{non_terminal} -> {' '.join(expansions)}")


[nltk_data] Downloading package punkt to /home/rosa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


VBD -> wanted
VBD -> slung
VBD -> met
NP -> NN NNS
PRP$ -> her
SBARQ -> WHADVP
NNS -> ladies
VBG -> pinching
NP -> NP PP
VBG -> casing
NN -> chapter
S -> S NP VP
JJR -> brighter
S -> ADVP NP
NN -> fat
NN -> business
JJ -> deep
NN -> mind
NN -> underwear
VP -> VBZ S
VP -> VBG S
VB -> listen
VBD -> stole
VB -> play
IN -> about
VBD -> needed
VP -> VBD NP-TMP
VB -> turn
JJ -> flat
ADJP -> RB JJ
NP -> PRP NNS
RBR -> more
VBD -> cost
VBN -> betrayed
VB -> try
UH -> see
VP -> VBP SBAR
VBP -> put
S -> NP NP VP
NP -> NP VP
NN -> lot
ADVP -> RBS
NP -> NN
RB -> after
VP -> VBD PP S
NP -> PRP NN NNS
NN -> activity
JJS -> finest
VB -> cut
ADVP -> CC
NP -> PRP$ NNS NN
VBP -> understand
VBZ -> has
NNS -> thieves
JJ -> afraid
NN -> skill
JJ -> much
NP -> FW FW NN NN
ADVP -> RB RB RB
NN -> friendship
VP -> VB NP NP
NNS -> feelings
PRP -> myself
NN -> cow
NN -> sandwich
VB -> hurt
VP -> VB PRT
PRN -> SQ
VBZ -> knows
S -> VP
IN -> beyond
VBG -> collecting
SBARQ -> WHNP SQ
NN -> light
VP -> VB NP PRT ADVP